In [7]:
# Import libraries

from bs4 import BeautifulSoup
import urllib.request as ureq
from urllib.request import urlopen
import urllib.parse, urllib.error
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import pandas as pd
import numpy as np
import time

In [8]:
# Define search parameters

searched_words = ['Data-Scientist-jobs','Data-Science-jobs','Data-Analyst-jobs','Data-Engineer-jobs','Data-Analytics-jobs', 'Analytics-jobs','Data-Insights-jobs','Big-Data-jobs','Business-Intelligence-jobs','Data-Warehouse-jobs','Data-Consultant-jobs']
seek_base_url = 'https://www.seek.com.au/'
pages = list(range(1,10))

# Concatenate base_url, searched word values and page numbers to define the url for all pages

searched_url_all = [str(seek_base_url) + words + '/in-All-Melbourne-VIC?page=' + str(i) for i in pages for words in searched_words]
searched_url_150k = [str(seek_base_url) + words + '/in-All-Melbourne-VIC?page=' + str(i) + '&salaryrange=150000-999999&salarytype=annual' for i in pages for words in searched_words]
searched_url_100k = [str(seek_base_url) + words + '/in-All-Melbourne-VIC?page=' + str(i) + '&salaryrange=100000-999999&salarytype=annual' for i in pages for words in searched_words]

In [246]:
# Scrape searched urls and save as BeautifulSoup objects

soup_objects = []

for urls in searched_url_100k:
    soup = BeautifulSoup(ureq.urlopen(urls), "lxml")
    soup_objects.append(soup)

In [247]:
# Extract all job ids and append to job_ids

job_ids = []

for soup in soup_objects:
    job_id = Selector(text=soup.prettify()).xpath('//article/@data-job-id').extract()
    
    for ids in job_id:
        job_ids.append(ids)
len(job_ids)

1286

In [248]:
# Extract individual urls for each job_id

job_base_url = 'https://www.seek.com.au/job/'

# Concatenate base_url and job_id values to define the url for each job ad

job_id_url = [job_base_url + x for x in job_ids]

len(job_id_url)

1286

In [249]:
# Extract soup objects from each job_id

job_soup_objects = []

for job_urls in job_id_url:
    soups = BeautifulSoup(ureq.urlopen(job_urls), "lxml")
    job_soup_objects.append(soups)
    #time.sleep(1)

len(job_soup_objects)

1286

In [251]:
# Extract required elements from all the individual job_id_url

job_advertiser_list = []
job_title_list = []
job_class_list = []
job_sub_class_list = []
job_type_list = []
job_location_list = []
job_post_date_list = []
job_salary_list = []
job_desc_list = []

for job_soup in job_soup_objects:
    
    ext_job_advertiser = Selector(text=job_soup.prettify()).xpath('//div/h2/span[2]/span/text()').extract()
    job_advertiser_list.append(ext_job_advertiser)
    
    ext_job_title = Selector(text=job_soup.prettify()).xpath('//span/span/h1/text()').extract()
    job_title_list.append(ext_job_title)
    
    ext_job_class = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/div[2]/dd/span/span/strong/text()').extract()
    job_class_list.append(ext_job_class)
    
    ext_job_sub_class = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/div/dd/span/span/span/text()').extract()
    job_sub_class_list.append(ext_job_sub_class)
    
    ext_job_type = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/dd[3]/span/span').extract()
    job_type_list.append(ext_job_type)
    
    ext_job_location = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/dd[2]/span/span/strong').extract()
    job_location_list.append(ext_job_location)
    
    ext_job_post_date = Selector(text=job_soup.prettify()).xpath('//article/div/div[2]/div/section/dl/dd[1]/span/span').extract()
    job_post_date_list.append(ext_job_post_date)
    
    ext_job_salary = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/div[1]/dd/span/span').extract()
    job_salary_list.append(ext_job_salary)
    
    ext_job_desc = Selector(text=job_soup.prettify()).xpath('//article/div/div/div/span/div/div/div/div/div').extract()
    job_desc_list.append(ext_job_desc)
    
print(len(job_advertiser_list))
print(len(job_title_list))
print(len(job_class_list))
print(len(job_sub_class_list))
print(len(job_type_list))
print(len(job_location_list))
print(len(job_post_date_list))
print(len(job_salary_list))
print(len(job_desc_list))

1286
1286
1286
1286
1286
1286
1286
1286
1286


In [252]:
# Create DataFrame for all, med, high searches

med_df = pd.DataFrame(job_ids, columns =['job_ids'])
med_df['job_advertiser'] = job_advertiser_list
med_df['job_title'] = job_title_list
med_df['job_class'] = job_class_list
med_df['job_sub_class'] = job_sub_class_list
med_df['job_type'] = job_type_list
med_df['job_location'] = job_location_list
med_df['job_post_date'] = job_post_date_list
med_df['job_salary'] = job_salary_list
med_df['job_desc'] = job_desc_list

med_df.head(2)

,job_ids,job_advertiser,job_title,job_class,job_sub_class,job_type,job_location,job_post_date,job_salary,job_desc
0,36617279,[\n SEEK\n ],[\n Data Scientist\n ...,[\n Science & Technology\n...,"[\n , \n ...","[<span class=""lwHBT6d"">\n Fu...","[<strong class=""lwHBT6d"">\n ...","[<span class=""lwHBT6d"">\n 3...","[<span class=""lwHBT6d"">\n B...","[<div class=""templatetext"">\n ..."
1,36823796,[\n Morgan Consulting\n ...,[\n Data Scientist -\n ...,[],"[\n , \n ...","[<span class=""lwHBT6d"">\n Co...","[<strong class=""lwHBT6d"">\n ...","[<span class=""lwHBT6d"">\n 2...","[<span class="""">\n <strong ...","[<div class=""templatetext"">\n ..."


In [235]:
# Save high_df to csv

high_df.to_csv('C:/Users/Georgie/Desktop/DSI/project-four/high_df.csv')

In [245]:
# Save all_df to csv
all_df.to_csv('C:/Users/Georgie/Desktop/DSI/project-four/all_df.csv')

In [253]:
# Save med_df to csv
med_df.to_csv('C:/Users/Georgie/Desktop/DSI/project-four/med_df.csv')

In [ ]:
## The end - please refer to Part 2 notebook